In [18]:
import pandas as pd

# 1. Lire le fichier CSV avec encodage Windows-1252
df = pd.read_csv('medicaments_ecrasables_input.csv', header=0, sep=';', encoding='cp1252')
df.astype('str').dtypes

# 2. Supprimer les colonnes "Classe ATC" et "Libellé ATC3"
df.drop(columns=['Classe ATC', 'Libellé ATC3'], inplace=True)
df.drop(df.columns[-1], axis=1, inplace=True)  # Supprime la dernière colonne

# 3. Remplacer les valeurs dans la colonne "Goût"
df['Goût'] = df['Goût'].replace({'G': 'Goût désagréable'})

# 4. Remplacer les valeurs dans la colonne "Modalités de préparation"
df['Modalités de préparation'] = df['Modalités de préparation'].replace({
    'D': 'Délitement possible',
    'E': 'Écrasement possible',
    'NE': 'Ne pas écraser ni croquer',
    'NO': 'Ne pas ouvrir',
    'O': 'Ouverture possible'
})

# 5. Remplacement lettre par lettre dans la colonne de méthode d'administration
remplacement_admin = {
    'C': 'prise possible avec de la compote<br>',
    'E': 'prise possible avec de l’eau<br>',
    'J': 'prise possible avec un jus d’orange<br>',
    'L': 'prise possible avec du lait<br>',
    'V': 'prise immédiate après préparation<br>',
    'Y': 'prise possible avec un yaourt<br>'
}

def remplacer_codes_admin(valeur):
    if pd.isna(valeur):
        return valeur
    codes = [code.strip() for code in str(valeur).split(',')]
    traductions = [remplacement_admin.get(code, code) for code in codes]
    return ''.join(traductions)

df["Méthode d'administration si médicament écrasé, ouvert ou délité"] = df["Méthode d'administration si médicament écrasé, ouvert ou délité"].apply(remplacer_codes_admin)

# 6. Trier le tableau selon la colonne "Principe actif" et renommer
df.sort_values(by='Principe actif', inplace=True)

df.columns = [
    "Principe actif",
    "Galénique",
    "Classe",
    "Goût",
    "Préparation",
    "Sources",
    "Autres sources",
    "Administration du médicament modifié",
    "Galéniques alternatives",
    "Informations",
    "Réponses des laboratoires",
    "Date"
]

# 7. Afficher une prévisualisation
print(df.head())

# 8. Sauvegarder le fichier en UTF-8
df.to_csv('../assets/data/medicaments-ecrasables.csv', index=False, sep=';', lineterminator='\n')
print('\n Terminé. Exporté dans le dossier /assets/data/ .')


                                      Principe actif  Galénique  \
0  ABACAVIR / DOLUTEGRAVIR / LAMIVUDINE 600/50/30...   Comprimé   
1  ABACAVIR / LAMIVUDINE / ZIDOVUDINE 300/150/300...  Comprimé    
2          ABACAVIR / LAMIVUDINE 600/300 mg (Kivexa)   Comprimé   
3                           ABACAVIR 300 mg (Ziagen)   Comprimé   
4           ABEMACICLIB 50, 100 et 150mg (Verzenios)   Comprimé   

                                              Classe              Goût  \
0  Antiviraux pour le traitement des infections a...               NaN   
1  Antiviraux pour le traitement des infections a...               NaN   
2  Antiviraux pour le traitement des infections a...               NaN   
3  Inhibiteurs de la transcriptase inverse nucléo...               NaN   
4                     Inhibiteurs de protéine kinase  Goût désagréable   

                 Préparation         Sources  \
0  Ne pas écraser ni croquer  Notice patient   
1  Ne pas écraser ni croquer  Notice patient   
2  Ne pa